In [ ]:
# Mounting Google Drive and Setting Dataset Path

from google.colab import drive
drive.mount('/content/drive')


import os
dataset_root = r'/content/drive/MyDrive/YOLO_Data/archive'
SUBFOLDER_NAME = 'bone fracture detection.v4-v4.yolov8'

FULL_DATASET_PATH = os.path.join(dataset_root, SUBFOLDER_NAME)
print(f"Dataset Root Set To: {FULL_DATASET_PATH}")

Mounted at /content/drive
Dataset Root Set To: /content/drive/MyDrive/YOLO_Data/archive/bone fracture detection.v4-v4.yolov8


# Training YOLOv8 Model on Bone Fracture Dataset


In [ ]:
import os
from ultralytics import YOLO
from google.colab import drive



dataset_root = r'/content/drive/MyDrive/archive'
SUBFOLDER_NAME = 'bone fracture detection.v4-v4.yolov8'
FULL_DATASET_PATH = os.path.join(dataset_root, SUBFOLDER_NAME)

print(f"Dataset Root Set To: {FULL_DATASET_PATH}")

def train_model():


    yaml_path = os.path.join(FULL_DATASET_PATH, "data.yaml")

    if not os.path.exists(yaml_path):
        print(f"\nERROR: Configuration file not found at {yaml_path}.")
        print("Please verify the full path of the 'archive' folder in your Drive.")
        return
    
    print(f"\nConfiguration file found. Starting training for 20 epochs...")

    model = YOLO('yolov8n.pt')
    model.train(
        data=yaml_path,
        epochs=20,
        imgsz=640,
        batch=16,
        name='fracture_detection_colab_20e',
        project='/content/drive/MyDrive/YOLO_Results',
        device=0
    )

    print("\nTraining Complete! Check the 'YOLO_Results' folder in your Drive for weights and metrics.")

if __name__ == '__main__':
    train_model()

Dataset Root Set To: /content/drive/MyDrive/archive/bone fracture detection.v4-v4.yolov8

Configuration file found. Starting training for 20 epochs...
Ultralytics 8.3.231 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/archive/bone fracture detection.v4-v4.yolov8/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov

# Testing and Validating the Trained YOLOv8 Model


In [ ]:
import os
import glob
from ultralytics import YOLO
from google.colab import drive



print("Mounting Google Drive...")


if not os.path.exists('/content/drive'): drive.mount('/content/drive')
dataset_root = r'/content/drive/MyDrive/archive'
SUBFOLDER_NAME = 'bone fracture detection.v4-v4.yolov8'


FULL_DATASET_PATH = os.path.join(dataset_root, SUBFOLDER_NAME)
SAVED_MODEL_PATH = '/content/drive/MyDrive/YOLO_Results/fracture_detection_colab_20e/weights/best.pt'

TEST_IMAGES_DIR = os.path.join(FULL_DATASET_PATH, 'test', 'images')
YAML_PATH = os.path.join(FULL_DATASET_PATH, "data.yaml")

print(f"Dataset Root Set To: {FULL_DATASET_PATH}")

def test_model():
    if not os.path.exists(SAVED_MODEL_PATH):
        print(f"\nERROR: Trained model weights not found at: {SAVED_MODEL_PATH}")
        print("Please check the 'YOLO_Results' folder in your Drive to confirm the path.")
        return

    print(f"\nLoading trained model from: {SAVED_MODEL_PATH}")
    model = YOLO(SAVED_MODEL_PATH)
    print("\n--- Starting Inference (Prediction) ---")
    test_files = glob.glob(os.path.join(TEST_IMAGES_DIR, '*.jpg'))

    if not test_files:
        print(f"WARNING: No JPG images found in the test directory: {TEST_IMAGES_DIR}")
        print("Skipping inference, proceeding to validation...")

    else:
        print(f"Running inference on {len(test_files)} test images...")
        results = model.predict(
            source=test_files,
            conf=0.25,
            save=True,
            project='/content/drive/MyDrive/YOLO_Test_Results',
            name='inference_on_test_set',
            imgsz=640,
            device=0
        )

        print("\nPrediction Complete!")
        print("Check your Google Drive for predicted images with bounding boxes:")
        print("/content/drive/MyDrive/YOLO_Test_Results/inference_on_test_set")

    print("\n--- Starting Validation (Quantitative Metrics) ---")
    if not os.path.exists(YAML_PATH):
        print(f"ERROR: data.yaml not found at {YAML_PATH}. Cannot run validation.")
        return

    metrics = model.val(data=YAML_PATH)
    print("\n--- Validation Metrics Summary ---")
    print(f"Metrics saved to: {metrics.save_dir}")
    print(f"  - mAP50 (Mean Average Precision at IoU 0.5): {metrics.box.map50:.4f}")
    print(f"  - mAP50-95 (mAP across IoU 0.5 to 0.95): {metrics.box.map:.4f}")
    print("  - Loss/Metrics plots, Confusion Matrix, and other results saved to this directory.")

    print("\nTesting and Validation workflow finished.")

if __name__ == '__main__': test_model()

Mounting Google Drive...
Dataset Root Set To: /content/drive/MyDrive/archive/bone fracture detection.v4-v4.yolov8

✅ Loading trained model from: /content/drive/MyDrive/YOLO_Results/fracture_detection_colab_20e/weights/best.pt

--- Starting Inference (Prediction) ---
Running inference on 169 test images...

0: 640x640 (no detections), 3.3ms
1: 640x640 (no detections), 3.3ms
2: 640x640 (no detections), 3.3ms
3: 640x640 (no detections), 3.3ms
4: 640x640 (no detections), 3.3ms
5: 640x640 (no detections), 3.3ms
6: 640x640 1 forearm fracture, 3.3ms
7: 640x640 (no detections), 3.3ms
8: 640x640 (no detections), 3.3ms
9: 640x640 (no detections), 3.3ms
10: 640x640 (no detections), 3.3ms
11: 640x640 (no detections), 3.3ms
12: 640x640 (no detections), 3.3ms
13: 640x640 1 shoulder fracture, 3.3ms
14: 640x640 (no detections), 3.3ms
15: 640x640 (no detections), 3.3ms
16: 640x640 (no detections), 3.3ms
17: 640x640 (no detections), 3.3ms
18: 640x640 (no detections), 3.3ms
19: 640x640 1 shoulder fractur